In [12]:
import cv2
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
from scipy import ndimage

def bogieBV(img):
    b,g,r = cv2.split(img)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    contrast_enhanced_green_fundus = clahe.apply(g)

     # applying alternate sequential filtering (3 times closing opening)
    r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
    r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
    r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
    R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1) 
    f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
    f5 = clahe.apply(f4)

    kernel = np.ones((3,3),np.uint8)
    dilation = cv2.dilate(f5,kernel,iterations = 1)
    blur = cv2.GaussianBlur(dilation,(3,3),0)
    kernels = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(blur, cv2.MORPH_OPEN, kernel)
    blur1 = cv2.GaussianBlur(opening,(5,5),0)
    full = cv2.morphologyEx(blur1, cv2.MORPH_OPEN, kernel)
    erosion_img = cv2.erode(full,kernel,iterations = 1)
    dilation_img = cv2.dilate(erosion_img,kernel,iterations = 1)
    erosion_img1 = cv2.erode(dilation_img,kernels,iterations = 1)
    closing = cv2.morphologyEx(erosion_img1, cv2.MORPH_CLOSE, kernel)
    ret,thresh2 = cv2.threshold(closing,25,255,cv2.THRESH_BINARY_INV)
    final =255-thresh2


    #plt.imshow(final,cmap ='gray')
    #plt.show()
    #cv2.imwrite('bogie5.jpg',final)
    return final

In [10]:
import glob
#=========USER START================
#folder path
path = 'RAW_FUNDUS_INPUT/*.jpg'
save_path = 'bv image bogie/'
#=========USER END==================

In [13]:
image_list = []
for filename in glob.glob(path):
    image_list.append(filename)
for i in image_list:
    c = cv2.imread(i)
    #=========PUT YOUR CODE HERE==========
    
    
    x = bogieBV(c)

    cv2.imwrite(save_path+ i.replace(path.replace("/*.jpg","")+"\\",""),x)